In [3]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# 1. Ellenőrizzük, hogy a transformers most már látja-e a torch-ot
print(f"Torch verzió: {torch.__version__}")

model_id = "NYTK/PULI-GPT-2"

# 2. Modell betöltése (félprecizitással a memóriatakarékosságért)
print("PULI-GPTrio betöltése, ez eltarthat egy ideig...")
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    dtype=torch.float16, 
    low_cpu_mem_usage=True,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)


Torch verzió: 2.9.1+cpu
PULI-GPTrio betöltése, ez eltarthat egy ideig...


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Error while downloading from https://huggingface.co/NYTK/PULI-GPT-2/resolve/main/pytorch_model.bin: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.
Trying to resume download...
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [4]:
# 3. A generátor létrehozása
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)


Device set to use cpu


In [5]:

# 4. Kísérleti propaganda szöveg generálása
# Olyan promptot adunk, ami a korábbi 10/12-es teszted stílusát idézi
prompt = "A brüsszeli bürokraták legújabb döntése világosan megmutatja,"

print("\nGenerált szöveg:")
print("-" * 30)
output = generator(prompt, max_length=150, repetition_penalty=1.2, do_sample=True, temperature=0.8)
print(output[0]["generated_text"])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.



Generált szöveg:
------------------------------


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


A brüsszeli bürokraták legújabb döntése világosan megmutatja, hogy a hagyományos európai értékrendet követő új pártok nem tudják majd betölteni azt az űrt, ami a bevándorlás-ellenes baloldali populizmus és a liberális demokrácia között tátong. A magyar kormányt már nem érdekli a kontinens sorsa, a liberálisok pedig nem számíthatnak politikai szövetségesekre.
Trócsányi László igazságügyi miniszter – aki a tervek szerint május végéig tölti be mandátumát – a Soros Alapítványnak adott interjújában arról beszélt: a Soros György elleni támadás újabb jele annak, hogy Európa elvesztette identitását és erejét. A miniszter elmondta, hogy az Európai Unió az elmúlt időszakban arra volt kénytelen ráébredni, hogy az illegális migráció és a klímaváltozás olyan pusztító erővel bírnak, hogy meg kell állítania önmagát. Ezért az európai intézményeknek ismét fel kellett vállalniuk önmagukat és meg kellett találniuk azokat a mechanizmusokat, amelyek képesek kezelni ezeket a kihívásokat. Ehhez pedig szükség

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cpu


In [7]:
import pandas as pd
csv_file = r"../../data/propaganda_articles.csv"

df = pd.read_csv(csv_file, encoding="utf-8-sig")

In [ ]:
df = df[df['label'] == 1][['title', 'text']]

In [13]:
from datasets import Dataset
import os
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# 2. Adatok tisztítása és előkészítése
# Csak a label=1 sorok
df_prop = df[df['label'] == 1].copy()

# A HIBA JAVÍTÁSA ITT TÖRTÉNIK:
# 1. Kitöltjük a hiányzó értékeket üres stringgel ("")
df_prop['title'] = df_prop['title'].fillna("").astype(str)
df_prop['text'] = df_prop['text'].fillna("").astype(str)

# 2. Összefűzés
df_prop['combined_text'] = df_prop['title'] + " \n " + df_prop['text']

# 3. Biztonsági szűrés: dobjuk ki, ami esetleg üres maradt
df_prop = df_prop[df_prop['combined_text'].str.strip().str.len() > 10]

print(f"Tanítás indul {len(df_prop)} db cikkel...")

dataset = Dataset.from_pandas(df_prop[['combined_text']])

# 3. Tokenizálás
def tokenize_function(examples):
    return tokenizer(
        examples['combined_text'], 
        truncation=True, 
        padding='max_length', 
        max_length=128
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)

Tanítás indul 698 db cikkel...


Map: 100%|██████████| 698/698 [00:00<00:00, 2476.84 examples/s]


In [14]:
import os
from transformers import Trainer, TrainingArguments


data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir='./propaganda_model',
    num_train_epochs=1,
    per_device_train_batch_size=1,  # VEDD LE 1-RE, hogy ne fagyjon le a gép!
    gradient_accumulation_steps=4,  # Ez szimulálja, mintha 4-es batch size-od lenne
    logging_dir='./logs',
    logging_steps=10,
    save_steps=100,
    learning_rate=5e-5,
    weight_decay=0.01,
    report_to=[],
    use_cpu=True # Mivel korábban láttuk, hogy nincs GPU-d
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

print("A finomhangolás elindult... Ez CPU-n sokáig fog tartani!")
trainer.train()
trainer.save_model('./propaganda_model_final')

The model is already on multiple devices. Skipping the move to device specified in `args`.


A finomhangolás elindult... Ez CPU-n sokáig fog tartani!


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
output = generator(
        prompt,
        max_length=150,          # Milyen hosszú legyen (szavakban/tokenekben)
        num_return_sequences=1,  # Hány verziót írjon
        do_sample=True,          # Legyen kreatív (ne mindig ugyanazt)
        temperature=0.85,        # Kreativitás mértéke (0.7-0.9 a legjobb)
        repetition_penalty=1.3,  # Hogy ne ismételje önmagát (nagyon fontos GPT-2-nél!)
        top_k=50,
        top_p=0.95,
        truncation=True
    )
print(output)